# Geospatial Data in Python with GeoPandas 

## Introduction

The goal of this notebook is to give you a **tip of the iceberg introduction** to working with geospatial data in Python using the **geopandas** package.  The sample data and use cases are related to a UC Berkeley research a project that I have been working on called [The Louisiana Slave Conspiracies](https://dlab.berkeley.edu/landing-page/louisiana-slave-conspiracies). This project explores several slave conspiracies that occured in colonial Louisiana during the late 1700s and early 1800s. Since very little data exist for this time period, we begin with an exploration of US Census data from the early 1800s, shortly after the Louisiana Purchase made the Louisiana and Orleans Territories part of the United States.

> ### Assumptions

> This tutorial assumptions you have basic working knowledge of Python and of geospatial data.   If you need a geospatial refresher, we can start with this a **very** [Brief Introduction to Geospatial Data](https://docs.google.com/presentation/d/1d9GNcLDsnLxfLmrNRNZE976sHN5qNfkU9Rl2gabUsWc/edit?usp=sharing).

 
 
## GeoPandas and related Geospatial Packages

[GeoPandas](http://geopandas.org/) is a relatively new package that makes it easier to work with geospatial data in Python. In the last few years it has grown more powerful and stable. This really is great because previously it was quite complex to work with geospatial data in Python.  GeoPandas is now the go to package for working with geospatial data. 

`GeoPandas` provides convenient, unified access to the functionality of the [pandas](https://pandas.pydata.org/) package , extending it with the geospatial processing capabilities provided by a number of lower level spatial data packages including [shapely](https://pypi.python.org/pypi/Shapely) for geometry processing, [fiona](https://pypi.python.org/pypi/Fiona) and [GDAL/Ogr](https://gdal.org) for spatial data file IO and[ pyproj](https://github.com/jswhit/pyproj) and [PROJ.4](https://github.com/OSGeo/proj.4/wiki) for map projections and coordinate systems.  


We will also use a few other optional geospatial libraries that are  commonly used with geopandas, including:

- **rtree** for spatial indexing to improve performance
- **geopy** for geocoding and for geodesic distance calculations
- **pysal** for spatial analysis functions such as data classification methods.
- **descartes** for ploting Shapelygeometric objects with Matplotlib.


Finally, we will use a number of standard Python libraries including pandas, numpy, and matplotlib.


# Setup

Installing Geopandas can be a bit complex due to the libraries that it depends on.  See the [Geopandas documentation ](http://geopandas.org/install.html) for help with this process - read it carefully as that will save you many headaches!

We will use the [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) Jupyter notebook environment for this workshop so that we will all have the same working enviroment. This is a *freemium* tool that requires a Google account.

## Open the workshop notebook

To get started, open this notebook and follow along:

- Google Colaboratory Notebook URL: http://bit.ly/geopandas_sp2019

Once you open the notebook, click **File > Save a copy in Drive** to make a copy of the notebook that you can edit. This will be saved to a Google Drive Collaboratory folder in your Google account.



## Install Geopandas and dependencies

Google Colaboratory comes with a Juypyter notebook environment with the most common Python packages already installed. To import a library that's not installed by default, you can use **!pip install** or **!apt-get install**.

<br>

To get started with geopandas, execute but do not change the code in the following cell. (*The install process we will follow is from [this notebook](https://colab.research.google.com/drive/1tSmJmjD3sTI31Cg1UCIKiE10dBUmWUG7#scrollTo=wHnmdr_QkKec&forceEdit=true&offline=true&sandboxMode=true)*)

>**IMPORTANT** -  if you are installing these Python packages on your local computer see [Geopandas documentation ](http://geopandas.org/install.html) . Do not use the code below as this is for the Google Colaboratory environment.

>> If you have your geopandas enviroment installed locally, you can get the data and notebook for this tutorial are in this github repository: https://github.com/dlab-geo/geopandas_intro



In [0]:
#######################################################
# Code to install geopandas in Google Colaboratory
# You need to run this code each time you run this 
# notebook on Google Colab
# Should take about 1.5 minutes.
#######################################################
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install pysal
!pip install pysal
# Install mapclassify
!pip install mapclassify
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

## Import GeoPandas and Related Libraries

We will start by importing the libraries that we will use.


In [0]:
import pandas as pd
import geopandas as gpd
import mapclassify
import matplotlib.pyplot as plt
from shapely.geometry import Point


## About the Data 

This tutorial uses historical census data for the USA and the Orleans Territory, most of which is now called Louisiana, that were obtained from the `NHGIS`, or *National Historical Geographic Information System* website ([IPUMS NHGIS, University of Minnesota, www.nhgis.org](https://www.nhgis.org)).  A cartographic boundary file for the United States was obtained from the [US Census website](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).


## Fetch the Data with `wget`

The data and related notebooks for this tutorial are in this github repository: https://github.com/dlab-geo/geopandas_intro


However, to keep things simple in the GC environment we will use the command `wget` to fetch the data frthat repo and store it in the `Google Collabortory` environment for the duration of this session.


In [0]:
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/us_states.zip'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/uscounties_1810.zip'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/ne_us_major_rivers.geojson'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/orleans_census_data1810.csv'
!wget 'https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/lsc_points.csv'


### Take a look at the data files

Take a look at the data we will use with the **ls** command (on mac) or the **dir()** command (on windows).

In [0]:
!ls

## Reading in a Spatial Data

GeoPandas makes it easy to read in almost any kind of vector data file including the [ESRI Shapefile](https://en.wikipedia.org/wiki/Shapefile) with the [read_file](http://geopandas.org/io.html) command.  You simply put the name of the file in quotes and assign the resulting object to a simple yet informative variable name, here `usa1810`.

You can see from the previous cell that the `usa1810` shapefile is compressed in a zipfile. We can load the zipfile directly in Geopandas.

In [0]:
usa1810 = gpd.read_file("zip://./uscounties_1810.zip")  #US counties in 1810

Sometimes geopandas cannot read a zipped shapefile. If this is the case, unzip it and read it in directly.

In [0]:
# Unzip the shapefile
!unzip ./uscounties_1810.zip

# Read shapefile into a geopandas geodataframe object.
usa1810 = gpd.read_file("./uscounties_1810.shp")  #US counties in 1810

### Discussion - what is a Shapefile?


## GeoDataFrame

The `gpd.read_file` command returns a geopandas **GeoDataFrame** object.



In [0]:
type(usa1810)

The `geodataframe` is a tabular pandas dataframe with extra geospatial capabilities. First let's take a look at the data frame using the **head** method.

In [0]:
usa1810.head()

Because it is a pandas dataframe you can use all the pandas data frame methods with it.  Some examples are shown below (in case you don't believe me!)

In [0]:
usa1810.SHAPE_AREA.sum() / (1000 * 1000)  # What wass the area in square kilometers of the USA in 1810?
#usa1810.STATENAM.nunique()  # How many states or territories did it have?
#usa1810[usa1810.SHAPE_AREA / (1000*1000) > 25000].STATENAM  # What states or territories where larger than 25,000 square kilometers?


### GeoSeries

You will notice that the `usa1810` geodataframe has special column named **geometry**. This column is of type *GeoSeries*, taking its name and its base functionality from the pandas series object. 


In [0]:
type(usa1810.geometry)


The GeoSeries extends this with the attributes and methods from the [Shapely](https://shapely.readthedocs.io) geometry library. 

Let's take a look at the geometry a bit more closely by considering the row for New York County, New York.

In [0]:
type(usa1810[usa1810['NHGISNAM']=='New York'].geometry)

A series can have one or more items and includes the row index, column value and data type.

In [0]:
usa1810[usa1810['NHGISNAM']=='New York'].geometry

The geometry column returns a geoseries even if only one row is requested. But many operations in geopandas want to work on the geometry not the series object. We can reference the geometry by the row index, which is shown above. When executed in a cell this will display the geometry shape.

In [0]:
usa1810.geometry[414]

If we print the geometry we see the [well-known text representation](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry), or WKT.

In [0]:
print(usa1810.geometry[414])

It's a pain to reference column values by row index. It's easier to select a row or row based on the value in one of the columns. When we do this, we can useuse the numpy **squeeze** method to extract the geometry value - *if the returned geoseries only contains one item.*

 Let's display the geometry for the county of New York, New York.

In [0]:
usa1810[usa1810['NHGISNAM']=='New York'].geometry.squeeze()


What if we try to display more than one geometry - in other words all the geometries in a `geoseries`?



In [0]:
usa1810[usa1810['STATENAM']=='New York'].geometry

# Does the squeeze() method help us here?
# usa1810[usa1810['STATENAM']=='New York'].geometry.squeeze()

## Summary

When working in geopandas, you have attributes and methods for **geodataframes**, **geoseries** and **geometry** objects.

You can use the "dot-tab" command to what is available for each type of  geospatial object. This is a great way to explore the data, when used along with the help page and the GeoPandas online documentation.

Uncoment the rows below to give it a try.

In [0]:
# Uncomment each of the lines below, one at a time. 
# Then press tab after the dot to see available methods and attributes

## for a geometry
#x = usa1810.geometry[414]
#x.

## for a geoseries
#usa1810.geometry.

## for a geodataframe
#usa1810.

## Mapping GeoDataFrames

We can use the geopandas **plot** method to display multiple geometries in a geodataframe or geoseries. This uses `matplotlib` and the matplotlib `pyplot` module under the hood.

In [0]:
usa1810.plot()  # it's really that simple!

We can also plot a subset of the geodataframe.

In [0]:
# Plot all the 1810 counties in New York state
usa1810[usa1810['STATENAM']=='New York'].plot()

And we can plot a geoseries with plot()

In [0]:
# plot the geometry geoseries
usa1810[usa1810['STATENAM']=='New York'].geometry.plot()

Pretty cool to be able to make a map with a single command. However, there is always room for improvement. You can find out more about the plotting options for basic maps in the geopandas documentation and in the matplotlib documentation.

We can use some options to make a prettier map. Take a minute to consider what each option does.

In [0]:
usa1810.plot(linewidth=0.25, edgecolor='black', facecolor='pink',  figsize=(14,10), )

You can take a look at the method documentation for **plot** to see all of the available options.

In [0]:
#usa1810.plot?

## Challenge

Let's take a few minutes to practice some of what we have done so far with a different data set.

- Read the file **us_states.zip** into a geopandas dataframe named **usa**.
- Take a look at the data in this dataframe using `head`.
- Then, make a map of the `usa`, 
    - setting the `figsize` to (14,10)
    - the fill color to green,
    - and the outline color to white

In [0]:
# your code here to load the data from the zip file into a geodataframe


In [0]:
# your code here to plot the geodataframe

## Challenge - solution

In [0]:
usa = gpd.read_file('zip://./us_states.zip')
usa.head()

In [0]:
usa.plot(linewidth=0.25, edgecolor='white', facecolor='green',figsize=(14,10))

## Spatial Subsetting
Well, that map of `us_states` is not great - why? 

Let's zoom in on the continental USA by spatially subsetting the data with the geopandas **cx** method.  This method takes the form:
>usa1810.cx[xmin:xmax, ymin:ymax]

>where:
- **xmin** is the minimum x coordinate value
- **xmax** is the maximum x coordinate value
- **ymin** is the minimum y coordinate value
- **ymax** is the maximum y coordinate value

Let's try that.

In [0]:
usa.cx[-130:-80, 25:45].plot(linewidth=0.25, edgecolor='white', facecolor='green',figsize=(14,10))

### Question

What exactly is **cx** doing?  Let's explore it a bit more. Change the minimum Y value to 30. Do Texas and Florida get clipped?

Take a second to uncomment the command below and read the documentation for `cx`. Then update the values in the previous code cell to get all states.

In [0]:
#usa.cx?

### Saving our spatial subset

We can make that subset permanent.

In [0]:
# FYI: conus is shorthand for continental USA
conus= usa.cx[-130:-50, 20:50].reset_index(drop=True)
conus.head()

In [0]:
# Plot the subset
conus.plot()

### Any questions?



---



## Map Overlays

A key strength of geospatial data analysis is the ability to overlay data that are located in the same coordinate space. Let's overlay the USA in 1810 on top of the USA in 2017 to visualize the change. Both of these data sets use the same coordinate reference system -  decimal degrees of latitude and longitude referenced to the **World Geodetic System of 1984**, or **WGS84** coordinate reference system (more about that in a minute).

We will explore two methods for doing this, as shown in the [GeoPandas documentation](http://geopandas.org/mapping.html)

The general process is as follows:

- First identify your base map - the layer to draw first, or at the bottom of the stack of layers.
- Then you add the next layer, referencing the base map as the **ax**. 

In [0]:
# Map the us states with the 1810 states and territories overlayed.
base = conus.plot(color='white', edgecolor='black',  figsize=(14,10))
usa1810.plot(ax=base)

We can add even more layers. These will draw in the order that you add them. Consider the following code.

In [0]:
base = conus.plot(color='white', edgecolor='black',  figsize=(14,10))
usa1810.plot(ax=base)
conus.centroid.plot(ax=base, color="red")  # Hey - what's happening here?

We can get even fancier with our maps by using the more **matplotlib** options. To access these you need to import mapplotlib.

In [0]:
# Mapping with advanced matplotlib settings

fig, ax = plt.subplots(1, figsize=(14,10))  # Initialize the plot figure (drawing area) and axes (data area)
ax.set_aspect('equal')   # set the aspect ratio for the x and y axes to be equal. 
                         # This is done automatically in gdf.plot()
    
base = conus.plot(ax=ax, color='black')  # Set the base map, or bottom map layer
usa1810.plot(ax=base, color='white', edgecolor="black")  # draw the data with the base
_ = ax.axis('off') # Don't show the x, y axes and labels in the plot
ax.set_title("USA 2017 and 1810")  # Give the plot a title
plt.show()

### Questions?

If your lost, don't worry. Many folks, myself included, scratch their head and copy matplotlib code, amazed and mystified when it works. Gradually it sinks in. You can expediate the process by reviewing any of a number of good online tutorials, like this one from [DataCamp](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python).

## Geometric Aggregrations

We often receive data that has more detail than we need. For example, we might have population by county when we want it by state. In pandas you can use a `groupby` operation to aggregate the data values.  

With geospatial data we often obtain data that has more geometric detail than we need. We can use the geopandas **dissolve** method to aggregate geometric data that share a common value.

For example, let's create a single geometry that is the outline of the continental US by dissolving the state geometry in the `conus` geodataframe. 
Start by taking a look at the geodataframe again.


In [0]:
conus.head()

We need a column value to indicate how to aggregate the geometry. We can create this column and populate it if it doesn't exist. 


In [0]:
# create a column with a single value that we can dissolve on
conus['country'] = 'usa' 
conus.head()

Now we can use the **dissolve** method to merge all the states that have the value "usa" in the country column.

Note, **dissolve** won't know what to do with the values specific to each state - like STATE (name), GEOID, ABBREV, so we remove those columns first.

In [0]:
# Select on the columns we weant to keep in dissovled geodataframe
conus_outline = conus[["country","geometry"]]  

# dissolve the interior polygons
conus_outline = conus.dissolve(by='country', as_index=False)

# take a look
conus_outline.head()



Finally, plot the dissolved geodataframe.

In [0]:
conus_outline.plot()

## Challenge

1. Use **dissolve** to create a **usa1810_outline**
    - *Note: you do not need to add a new column - what column should you use to dissolve by?*


In [0]:
# your code here

## Challenge - solution

In [0]:
#usa1810.head()
# Select on the columns we weant to keep in dissovled geodataframe
usa1810_outline = usa1810[["DECADE","geometry"]]  

# dissolve the interior polygons
usa1810_outline = usa1810_outline.dissolve(by='DECADE', as_index=False)

# take a look
#usa1810_outline.head()
usa1810_outline.plot()

## Challenge

Create a new **usa1810_states** geodataframe by dissolving the usa1810 county geometry.

- *What column do you use to dissolve by?**

In [0]:
# your code here

## Challenge - solution

In [0]:
usa1810.head()
usa1810_states = usa1810[['STATENAM','geometry']]
usa1810_states = usa1810_states.dissolve(by='STATENAM', as_index=False)
usa1810_states.plot()

## Challenge 

Display the **usa1810_outline** on top of the **conus_outline**

- *HINT: You can do this quickly by copying related code from a previous section and changing the geodataframe names*

In [0]:
# your code here


# Coordinate Reference Systems (CRS) 

Did you notice anything funny about the **shape** of the USA as mapped above?  How does it differ from this?
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Map_of_USA_with_state_names.svg/640px-Map_of_USA_with_state_names.svg.png" width="800px"></img>


#### Why does the shape differ? 

Here's why:

<img src="http://tse3.mm.bing.net/th?id=OIP.lyDmHXX9VdoEOWDQlqppSAHaEy" width="500px"></img>

When we map data encoded with a spheriodal coordinates (longitude & latitude) on a 2D plane like a computer screen we get distortion!

## Map Projections and CRS Transformations

In order to reduce distortion in maps we apply a map projection (math) to transform 3D geographic coordinates to 2D projected map coordinates.
<img src="https://www.e-education.psu.edu/natureofgeoinfo/sites/www.e-education.psu.edu.natureofgeoinfo/files/image/projection.gif"></img>


CRS transformations are often necessary for geopandas geometric operations like area and distance calculations which assume a 2D plane.

## Defining and Transforming a CRS
The process for transforming a CRS is:

1. Make sure a **crs** is defined for the geopandas dataframe by checking the **crs** property. 
2. If it is not set, you can **define** it.
3. Transform the coordinate geometry to a new CRS using the **to_crs** method.
- This returns a new geodataframe with the new coordinate values and CRS.
- You need to know what CRS to use!!

In [0]:
# Check the CRS of our gdfs
print("The CRS of the conus geodata frame is: " + str(conus.crs))
print("The CRS of the usa1810 geodata frame is: " + str(usa1810.crs))


What's an **epsg:4326**?  That's an [EPSG](http://www.epsg.org/) code for the geographic CRS known as the [World Geodectic System of 1984](https://en.wikipedia.org/wiki/World_Geodetic_System#WGS84), or `WGS84`. This is the most commonly used CRS for latitude and longitude coordinate data and is the default CRS for most mapping software when the data does not have a defined CRS.

We can make our map look better by transforming it to a 2D projected `CRS`. A projected CRS applies a mathematical transformation to the data based on a [map projection](https://en.wikipedia.org/wiki/Map_projection)

Common map projections for USA data and their EPSG codes include:

- **Web Mercator** (epsg:3857)
- **USA Contiguious Albers** (epsg:7603)

Let's try these with our geodataframe.

First, let's transform the CRS of the **conus** geodataframe to Web Mercator. The **to_crs** method will return a new geododataframe.

In [0]:
# Transform geographic crs to web mercator - 3857
conus_3857 = conus.to_crs(epsg=3857)
conus_3857.plot()

## Challenge

Now you try it! Transform the **conus** geodataframe to one that uses the **USA Contiguous Albers CRS** (7603).

Then, display the output geodataframe.

In [0]:
# Your code here


## Challenge Solution

In [0]:
# Transform the conus geodataframe to USA Albers (7603)
conus_7603 = conus.to_crs(epsg=7603)
conus_7603.plot()

## Multiplots

Let's plot all the data in all 3 CRS together.

In [0]:
fig, ax = plt.subplots(ncols=3, figsize=(18,4), subplot_kw=dict(aspect='equal'))
# Don't show the coordinate axis
ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
# Show a title
ax[0].set_title('Geographic, unprojected (4326)')
ax[1].set_title('Web Mercator (3857)')
ax[2].set_title('Albers EA (7603)')
# display
conus.plot(ax=ax[0])
conus_3857.plot(ax=ax[1])
conus_7603.plot(ax=ax[2])
plt.show()

You can see we get very different maps of the USA depending on the CRS. Web Mercator is best suited for large geographic extents between +-60 degrees (i.e. distortion increases as you move away from equator and towards the poles). USA Contiguous Albers is best for the continental USA.  We could subset the data to remove Alaska, Hawaii and Puerto Rico.

### Important Notes

1. Data need to be in the same CRS in order to be mapped or analyzed together.
2. The units of a CRS are part of the CRS definition. These are typically decimal degrees for geographic (lat/lon) data and meters or feet for projected data.
3. It's not obvious what the best projected CRS is for your map or analysis. You need to review the recent literature (as these things change) and try different ones.  Here is a good starting place, [epsg.io](http://epsg.io/).


> A detailed discussion CRS and map projections is beyond the scope of this notebook. Understanding these, however, is **necessary** for working sucessfully with geospatial data! There are a number of online resources that can be found with a web search to help you get started.

**Any Questions?**


---



# Spatial Measurements

Geopandas uses the  [Shapely library](https://shapely.readthedocs.io/en/stable/manual.html) to compute spatial measurements like area and length for individual geometries or all the geometries in a geoseries.  The available measurements depend on the geometry type. For example, we can compute area and perimeter for polygons, length for lines, and distances between points.  Read the GeoPandas and Shapely documentation to get a sense of all the meausurements you can compute.

### Calculating Area

Let's compute the area of a single geometry in a geodataframe.

In [0]:
conus[conus['STATE']=='Montana'].geometry.area # returns a pandas series

# Use Squeeze() to return the value when the geoseries only has one element.
#conus[conus['STATE']=='Montana'].geometry.area.squeeze()  

**Question**  - What are the units for the above area value?

We can compute the area of all geometries in the geodataframe.

In [0]:
 conus.geometry.area

In [0]:
# We can also use the shorter syntax
conus[conus['STATE']=='Montana'].area

# or
#conus.area

Above, we dynamically calculated area. But we can also append it to the geodataframe.

In [0]:
# Update the dataframe
conus['area'] = conus.area
conus.head(15)

### Calculating Length or Perimeter

Similarly we can calculate the perimeter of our state polygons.

In [0]:
# Perimeter of the state of Montana
conus[conus['STATE']=='Montana'].length

# Perimeter of all states in the continental USA
#conus.length

### Calculating Distance
We can also compute the shortest distance between individual geometries.

In [0]:
conus[conus['STATE']=='Montana'].squeeze().geometry.distance(conus[conus['STATE']=='California'].squeeze().geometry)

## Spatial Measurements and CRSs

The output of spatial measurements depend on the CRS and is expressed in the units of the CRS. The Shapely library assumes a two dimensional planar coordinate system and makes no transformation on the data - that is left for the analyst.

Let's compare the output of the area calculations using the different CRSs.



In [0]:
print("Area of Montana")
print("- CRS WGS84: %.2f" % conus[conus['STATE']=='Montana'].geometry.area)
print("- CRS Web Mercator: %.2f" % conus_3857[conus_3857['STATE']=='Montana'].geometry.area)
print("- CRS Albers: %.2f" % conus_7603[conus_7603['STATE']=='Montana'].geometry.area)

**Question**

Which CRS returns the (most) correct value for the area of Montana? Check []Wikipedia](https://en.wikipedia.org/wiki/Montana)...

*Discussion*


## CRSs and Distance Calculations
Let's compute the distance between Montana & California using the Web Mercator and Albers CRSs.

In [0]:
# Your code here
wm_dist_m = conus_3857[conus_3857['STATE']=='Washington'].squeeze().geometry.distance(conus_3857[conus_3857['STATE']=='California'].squeeze().geometry)
wm_dist_km = wm_dist_m / 1000
print("web mercator dist KM:", wm_dist_km)
#
al_dist_m = conus_7603[conus_7603['STATE']=='Washington'].squeeze().geometry.distance(conus_7603[conus_7603['STATE']=='California'].squeeze().geometry)
al_dist_km = al_dist_m / 1000
print("Albers dist KM:", al_dist_km)


 **Question**
 
 Which CRS returns the best result? Check against Google Maps to find out.

### Determinining the Best CRS

It's not obvious or easy to find the best CRS for your spatial analysis. It really depends on what you are trying to measure (area, distance, direction, shape) and the location of your study area. There is a theoretically unlimited number of CRSs to choose from.

However, only a few are typically used in any one area and you figure this out by reviewing the literature of geospatial analysis in your field and geographic methods and tools in general.


For example, the UTM coordinate system is good for regional area, distance and direaction calculations. The system includes a number of different CRSs - one for 60 different zones in the two hempisphers.  UTM sone 10 north and 11 north are often used for the geospatial analysis California data.  UTM zone 10N (epsg=32610) also includes most of Oregon & Washington. Let's use this CRS to calculate the distance between the centroids of these two states.

In [0]:
# First transform the conus data to utm 10n (32610)
conus_utm = conus.to_crs(epsg=32610)

# Calculate the distance in meters
wm_dist_m = conus_utm[conus_utm['STATE']=='Washington'].squeeze().geometry.centroid.distance(conus_utm[conus_utm['STATE']=='California'].squeeze().geometry.centroid)

# Convert to kilometers
wm_dist_km = wm_dist_m / 1000
print("web mercator dist KM:", wm_dist_km)

## Geodesic distance - DEMO

Over larger areas on the surface of the Earth it becomes difficult to impossible to find a good projected CRS for distance calculations. Geodesic distance calculations use a spheroidal model of the Earth to calculate the distance between to points and are thus more accurate.

We can use the **vincenty** function in the [geopy](https://geopy.readthedocs.io/en/stable/) library to compute geodesic distances quite accurately.

In [0]:
# Extrate the center points of Washington and California
wa_ctr = conus[conus['STATE']=='Washington'].centroid.squeeze()
ca_ctr = conus[conus['STATE']=='California'].centroid.squeeze()


In [0]:
# Create Point list objects
## IMPORTANT - what is the order of the coordinates?
wa_ctr_pt = [wa_ctr.y, wa_ctr.x]
ca_ctr_pt = [ca_ctr.y, ca_ctr.x]

In [0]:
# Use the vincenty function to compute the distance between these points
from geopy.distance import vincenty
vincenty(wa_ctr_pt, ca_ctr_pt).km

### Any Questions?

## Spatial Relationship queries

[Spatial relationship queries](https://en.wikipedia.org/wiki/Spatial_relation) consider how two geometries or sets of geometries relate to one another in space. Fortunately, these don't depend on the CRS of the data.

<img src="https://upload.wikimedia.org/wikipedia/commons/5/55/TopologicSpatialRelarions2.png" height="400px"></img>


Let's consider some spatial relationship queries between individual geometries and geopandas geodataframes / geoseries.

To start, let's create  geometry objects that represent the Parish of Pointe Coupee, Louisiana as a point and as a polygon.

In [0]:
# Polygon  representation of the Paris (county) of Pointe Coupee Louisiana in 1810
ptcoupee_poly = usa1810[usa1810['NHGISNAM']=='Pointe Coupee'].geometry.squeeze()
ptcoupee_poly



In [0]:
# Make a polygon representaton of Colonial Louisiana (Orleans Territory)
orleans_poly = usa1810_states[usa1810_states['STATENAM']=='Orleans Territory'].geometry.squeeze()

# And Louisiana today
la_poly = conus[conus['STATE']=='Louisiana'].geometry.squeeze()

In [0]:
# Point representation of the Paris (county) of Pointe Coupee Louisiana in 1810
ptcoupee_pt = usa1810[usa1810['NHGISNAM']=='Pointe Coupee'].geometry.centroid.squeeze()
ptcoupee_pt

In [0]:
# If we make these geometries a geoseries we can quickly plot together
gpd.GeoSeries([la_poly, orleans_poly, ptcoupee_poly, ptcoupee_pt]).plot(cmap="Reds")

Now let's consider the following spatial relationship query:

- Is Pointe Coupee within Orleans Territory?


Here is a list of the geopandas functions we can use to check spatial relationships.

*Which one should we use to answer the query?*

- equals
- contains
- crosses
- disjoint
- intersects
- overlaps
- touches
- within
- covers

In [0]:
# spatial relationship query
ptcoupee_poly.within(la_poly)


**Question** - How do you think we would  restate that using ** contains**?

We can test the spatial relationship between all geometries in a geodataframe.

In [0]:
usa1810_states.contains(ptcoupee_poly)

Since spatial functions return **True** or **False** we can use them to select only the rows that meet the condition being tested.

In [0]:
usa1810_states[usa1810_states.contains(ptcoupee_poly)]

## Challenge

Consider the following questions:

- What US states border Louisiana?

- Is Orleans Territory within Louisiana

Use the code cell below to try out some spatial relationship queries to answer those questions.

In [0]:
# Your code here

**Discussion**


##  Intersects - the most general spatial relationship query

The most useful, fastest and most general purpose spatial relationship query is **intersects**. You don't need to worry about selecting the correct spatial relationship predicate for your query or differences due to the resolution and alignment of your geometries.

Try it!

- Redo the previous queries with intersects.  
- Then try them again reversing the order of the objects.
    - Does order matter with intersects?

In [0]:
# your code here

### Questions?



---



## Geometric Processing

Spatial relationship queries return `True` or `False` when comparing geometries based on a spatial relationship predicate. These can be used to subset geodataframes.

Geometric processing operations, on the other hand, construct new geometries from one or more input geometries. We saw this earlier with the **dissolve** operation.

The [geopandas **overlay** operation](http://geopandas.org/set_operations.html) allows you to compare two geodataframes and return the set intersection, union, difference between them. 

Let's consider how we can use **overlay** to create a new geodataframe that represents the areas that were added to the continental United States after 1810.

*Note: these operations can be quite computationally intensive on complex geometries!*

In [0]:
# What states were added after Louisiana Purchase
usa_post1810 = gpd.overlay(conus, usa1810_outline, how='difference')

In [0]:
usa_post1810.head()


In [0]:
usa_post1810.plot()

## Question

How does the above geoprocessing output compare with that of the spatial relationship query code below:

In [0]:
conus[conus.disjoint(usa1810_outline.geometry.squeeze())].plot()

## Spatial Joins

Spatial join operations are used to join attribute data from one spatial object to another based on spatial location. In `geopandas` this is done with the **sjoin** operator.  Spatial joins borrow syntax from non-spatial attribute table joins.

Take a look at the documentation for **sjoin**.

In [0]:
#gpd.sjoin?

Let's explore spatial joins by:

- reading in a csv file of Louisiana slave conspiracies.
- making these data a geodataframe
- using a spatial join to add the parish in which the conspiracy took place to the slave conspiracy geodataframe.

In [0]:
# Read in a csv file that has lat/lon values
lsc_locs = gpd.read_file("./lsc_points.csv")
 
#take a look at the data
lsc_locs

Because we have read in a CSV file, we do not have data in the **geometry** column. We can use some code to add it.

In [0]:
# populate geometry column from lat/lon data
lsc_locs['geometry'] = lsc_locs.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis = 1)

# Take a look
lsc_locs

## SJOIN

We are now ready to use **sjoin** to add the parish (county) for each conspiracy.

In [0]:
lsc_with_parish = gpd.sjoin(lsc_locs, usa1810)

### CRSs must be the same for spatial joins.

First take a look at the crs of the lsc_locs data.

In [0]:
lsc_locs.crs

Why isn't the CRS defined?

## Defining a CRS

If you know the CRS of your data you can define it by referencing its **epsg** code. 

*Note: defining a CRS is not a transformation!*

In [0]:
lsc_locs.crs = {'init': 'epsg:4326'}

# If we know two geodataframes have the same CRS
# We can also define the CRS like this:
#lsc_locs.crs = usa1810.crs 

Now try that spatial join again!

In [0]:
lsc_with_parish = gpd.sjoin(lsc_locs, usa1810)


Take a look at the results...

In [0]:
lsc_with_parish.head()


## Attribute Joins

Attribute joins combine data from different tables based on a column with shared values.  Although these are not spatial they are widely used in geospatial analysis and in all data analysis.  We use the **merge** command for geopandas attribute joins.

<br>
Let's use an attribute join to simplfy the results of our spatial join.  Below, we take the geodataframe that resulted from our spatial join, **lsc_with_parish** and join the parish name - which is in the **NHGISNAM** column to our original **lsc_locs** geodataframe. The join is based on the fact that both tables have a **name** column that contains unique values for the slave conspiracy.

In [0]:
lsc_locs = lsc_locs.merge(lsc_with_parish[['name','NHGISNAM']], on='name', how="left", )
lsc_locs.head()

### Questions?



---



## Data Driven Mapping

Data driven mapping refers to the process of creating thematic maps by using data values to determine the symbology of mapped features - including their color, shape, size.  This is in contrast to setting the same symbology for all features as we have done above.

### Mapping categorical data
We can symbolize the color of our features by a categorical data value.

In [0]:
conus.plot(column="STATE", edgecolor="white")


### Mapping quantitative data

We can also color areas by quantitave data values. This is called a **choropleth** map. 

Below we map the1810 states and territorized colored by the values in the SHAPE_AREA column (area in square meters). This is a test example since this geodataframe does not have any more meaningful quantitative data columns.

In [0]:
usa1810.plot(column="SHAPE_AREA", cmap="Reds", edgecolor="grey")

### cmaps - colormaps

Note the use of the **cmap** option to set the matplotlib color palette for mapping the data.  Take a look at the [documumentation](https://matplotlib.org/users/colormaps.html) for these colormaps and rerun the previous code with a different value for **cmap**.   I strongly recommend that you read this documentation to improve your use of colormaps to effectively map data values. 

## Mapping Quantitative Data, Deep Dive

Let's bring in some population data about the 1810 parishes in Orleans Territory and use it to create quantitative data maps.

In [0]:

orleans_census1810 = pd.read_csv('./orleans_census_data1810.csv')
orleans_census1810.head()

Compare the **orleans_census1810** dataframe above with the **orleans** geopandas dataframe that we create below. What **columns** do these have in common?

In [0]:
orleans = usa1810[usa1810['STATENAM'] == 'Orleans Territory']

In [0]:
orleans.head()

Use the geopandas **merge** command to join the attributes of the pandas dataframe **orleans_census1810** to the **orleans** geodataframe by the values in the **GISJON** column.  Then take a look at the output which we call **orleans_popdata** below.

In [0]:
orleans_popdata = orleans.merge(orleans_census1810, on='GISJOIN')
orleans_popdata.head()

In [0]:
type(orleans_popdata)

We can now add options to the `plot` command to use data values to determine map symbology. Consider how this is done below with the data in the `nwslave_pop` (non-white slave population) column.

In [0]:
orleans_popdata.plot(column='nwslave_pop', cmap='Reds', edgecolor='black', legend=True, figsize=(8,6))

**Discussion**

Above, the plot option **column=** tells the plot command to use the values in the **nwslave_pop** column to determine the geometry colors based on the colormap specified by the **cmap** option. You can see the list of available [color maps here](https://matplotlib.org/users/colormaps.html). The full range of values in the `nwslave_pop` column is being scaled to the color palette called **Reds**.  This is called an `unclassified` or `classless` map. This map is a good first effort as it imposses no grouping on the data, thus making it easier to spot trends and outliers. But it is harder to interpret the data values within an area. 


A more common practice is to use a classification scheme to bin the data into classes and map those classes to a color palette. Let's try that below with `quantile` classification which is the most commonly used scheme when mapping data.

In [0]:

orleans_popdata.plot(column='nwslave_pop', cmap='Reds', edgecolor='black', 
                     legend=True, figsize=(8,6), scheme='quantiles')

Wow that gives a very different looking map!

## Challenge

In the code cell below recreate the above map with the classification schemes **equal_interval** and **fisher_jenks** to see how the look of the map changes.

> Bonus challenge: try a different colormap.

In [0]:
# Your code here

## Choropleth maps

The maps we just made are called `choropleth maps`. A [choropleth maps](https://en.wikipedia.org/wiki/Choropleth_map) is a data map that colors areas by data values.  This are the most common type of data map. It is also sometimes called a **heatmap**.

<br>
**HOWEVER**, when the areas being mapped vary in size it is not considered good cartographic practice to map **counts**.  Why do you think this is so?

Instead, choropleth maps typically symbolize ares by densites, ratios or rates that can compared across the different sized areas.

<br>

Let's map the ratio of non-white slaves (nwslave_pop) to free whites (white_pop).

In [0]:
# Create a new column that is the ratio of non-white slaves (nwslave_pop) to free whites (white_pop)
orleans_popdata['slave2white_ratio'] = orleans_popdata['nwslave_pop'] / orleans_popdata['white_pop']

# Map the ratio
orleans_popdata.plot(column='slave2white_ratio', cmap='Reds', edgecolor='black', 
                     legend=True, figsize=(8,6), scheme='quantiles')

We can get fancy with our choropleth maps...

In [0]:
fig, ax = plt.subplots(1, figsize=(12,12))

orleans_popdata.plot(ax=ax, column='slave2white_ratio', cmap='OrRd', edgecolor='black', legend=True, scheme='fisher_jenks')

for polygon, name in zip(orleans_popdata.geometry, orleans_popdata.COUNTY):
    ax.annotate(xy=(polygon.centroid.x, polygon.centroid.y), s=name)

_ = ax.axis('off')

ax.set_title("Ratio of Non-White Slaves to Free Whites, Orleans Territory, 1810")
plt.show()

Needless to say, labels are a bit tricky, regardless of the software you use to make a map!


## Question

In 1791 and 1795 two slave revolts where planned in the same parish in Orleans Territory. Although these plots involved different people and had different orgins, both were discovered and thwarted, leading to the trial and execution or emprisonment of many enslaved persons. Soon thereafter, the [German Coast Uprising of 1811](https://en.wikipedia.org/wiki/1811_German_Coast_uprising), which was the largest slave revolt in US history, occured in a different Orleans parish. Can you use the map above to guess the two parishes in which these three events occured? 

###  Any Final Questions?



---



## Going Further

### Start with the Package Documentation

- [GeoPandas Documentation](http://geopandas.org/)
- [Shapely Documentation(https://shapely.readthedocs.io/en/stable/)]

### I highly recommend this SciPy 2018 workshop on geopandas.

- The notebooks are [here](https://github.com/geopandas/scipy2018-geospatial-data).
- And be sure to watch the [youtube video](https://www.youtube.com/watch?v=kJXUUO5M4ok). 

### More Geopandas Practice

- Try this [Geopandas tutorial](https://www.datacamp.com/community/tutorials/geospatial-data-python) on plotting the path of Hurricane Florence.

### For a longer course

- Try the datacamp course [Visualizing Geospatial Data in Python](https://www.datacamp.com/courses/visualizing-geospatial-data-in-python), the first chapter is FREE!

### Interactive mapping 

- The [mplleaflet](https://github.com/jwass/mplleaflet) and [folium](https://github.com/python-visualization/folium) packages are very popular for creating interactive web maps in python notebooks. Check out the online documentation and do a web search for an online tutorial to get started.


# Thank you!


---

Last updated 02/27/2019 by Patty Frontiera (pattyf [at] berkeley [dot] edu)